<a href="https://colab.research.google.com/github/laresamdeola/Computer-Vision/blob/main/Parkinson's_Disease_Computer_Vision_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
import os
import shutil

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.listdir('Parkinson_Data')

['train_data', 'test_data', '.ipynb_checkpoints']

In [ ]:
shutil.rmtree('Parkinson_Data/.ipynb_checkpoints')

In [ ]:
os.listdir('Parkinson_Data')

['train_data', 'test_data']

In [ ]:
os.listdir('Parkinson_Data/train_data')
os.listdir('Parkinson_Data/test_data')

['pd', 'control', '.ipynb_checkpoints']

In [ ]:
shutil.rmtree('Parkinson_Data/train_data/.ipynb_checkpoints')

In [ ]:
shutil.rmtree('Parkinson_Data/test_data/.ipynb_checkpoints')

In [ ]:
os.listdir('Parkinson_Data/train_data')
os.listdir('Parkinson_Data/test_data')

['pd', 'control']

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_set = train_datagen.flow_from_directory(
    'Parkinson_Data/train_data',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

#

Found 34 images belonging to 2 classes.


In [ ]:
print(train_set.labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_set = test_datagen.flow_from_directory(
    'Parkinson_Data/test_data',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 8 images belonging to 2 classes.


In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               kernel_size=3,
                               activation='relu',
                               input_shape=[64,64,3]))

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
cnn.fit(x=train_set, validation_data=test_set, epochs=10)

Epoch 1/10
2/2 [==============================] - 2s 724ms/step - loss: 1.3329 - accuracy: 0.4412 - val_loss: 0.4964 - val_accuracy: 0.7500
Epoch 2/10
2/2 [==============================] - 1s 201ms/step - loss: 0.6816 - accuracy: 0.5882 - val_loss: 0.5994 - val_accuracy: 0.6250
Epoch 3/10
2/2 [==============================] - 1s 223ms/step - loss: 0.6970 - accuracy: 0.5882 - val_loss: 0.6803 - val_accuracy: 0.6250
Epoch 4/10
2/2 [==============================] - 1s 599ms/step - loss: 0.7084 - accuracy: 0.5294 - val_loss: 0.7202 - val_accuracy: 0.6250
Epoch 5/10
2/2 [==============================] - 0s 153ms/step - loss: 0.6960 - accuracy: 0.5588 - val_loss: 0.6205 - val_accuracy: 0.6250
Epoch 6/10
2/2 [==============================] - 0s 173ms/step - loss: 0.5480 - accuracy: 0.5882 - val_loss: 0.5292 - val_accuracy: 0.6250
Epoch 7/10
2/2 [==============================] - 0s 174ms/step - loss: 0.5598 - accuracy: 0.6471 - val_loss: 0.4908 - val_accuracy: 0.7500
Epoch 8/10
2/2 [====

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 30752)             0         
                                                                 
 dense (Dense)               (None, 128)               3936384   
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3937409 (15.02 MB)
Trainable params: 3937409 (15.02 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
# Credits:

# Lare Adeola, Celestine Iwendi, Abbas Aamir (2023).
# Creating a Sensor System for Safe Motor Navigation: HOMESWEET.
# Emerging Trends in IoT and Computing Technologies
# Proceedings of the International Conference on Emerging Trends
# in IoT and Computing Technologies, Chapter 79.
